# Detectron2 (Original+ResNeSt+centermask2)

<img src="https://dl.fbaipublicfiles.com/detectron2/Detectron2-Logo-Horz.png" width="500">

This is a modification of the offiical detectron2 colab that can be found [here](https://colab.research.google.com/drive/16jcaJoc6bCFAQ96jDe2HwtXj7BMD_-m5 ).

This colab is a combination of [facebookresearch/detectron2](https://github.com/facebookresearch/detectron2), [zhanghang1989/detectron2-ResNeSt](https://github.com/zhanghang1989/detectron2-ResNeSt) and [youngwanLEE/centermask2](https://github.com/youngwanLEE/centermask2).

My fork is located in [styler00dollar/Colab-Detectron2](https://github.com/styler00dollar/Colab-Detectron2).

You might need to edit some paths or parameter to fit your needs. This assumes you know what you are doing.


In [ ]:
#@title check gpu
!nvidia-smi

In [ ]:
#@title check amount of files in folder (specify path)
%cd "/path/"
!ls | wc -l

In [ ]:
#@title connect Google Drive if you want to copy dataset
from google.colab import drive
drive.mount('/content/drive')

# Install detectron2

In [ ]:
#@title Install dependencies
# install dependencies: (use cu101 because colab has CUDA 10.1)
!pip install -U torch==1.5 torchvision==0.6 -f https://download.pytorch.org/whl/cu101/torch_stable.html 
!pip install cython pyyaml==5.1
!pip install -U 'git+https://github.com/cocodataset/cocoapi.git#subdirectory=PythonAPI'
import torch, torchvision
print(torch.__version__, torch.cuda.is_available())
!gcc --version
# opencv is pre-installed on colab

# Install detectron2 or detectron2-ResNeSt

In [ ]:
#@title Install detectron2
%cd /content/
# install detectron2:
!pip install detectron2==0.3 -f https://dl.fbaipublicfiles.com/detectron2/wheels/cu101/torch1.5/index.html
# git clone detectron2
!git clone https://github.com/facebookresearch/detectron2

In [ ]:
#@title Install detectron2-ResNeSt
%cd /content/
# install detectron resnest
!python -m pip install 'git+https://github.com/zhanghang1989/detectron2-ResNeSt.git'
import detectron2

# clone detectron2 resnest
%cd /content/
!git clone https://github.com/zhanghang1989/detectron2-ResNeSt

In [ ]:
#@title Install detectron2 / clone centermask2
%cd /content/
# install detectron2:
!pip install detectron2==0.3 -f https://dl.fbaipublicfiles.com/detectron2/wheels/cu101/torch1.5/index.html
# git clone detectron2
!git clone https://github.com/youngwanLEE/centermask2

In [ ]:
#@title Import detectron2

# You may need to restart your runtime prior to this, to let your installation take effect
# Some basic setup:
# Setup detectron2 logger
import detectron2
from detectron2.utils.logger import setup_logger
setup_logger()

# import some common libraries
import numpy as np
import cv2
import random
from google.colab.patches import cv2_imshow

# import some common detectron2 utilities
from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog

# Train on a custom coco dataset

Copy a .7z file and extract its contents.

The datastructure should be like this:
 - some_name.7z
  - train
    - image1.png
    - lksjfdl.png
    - ...
    - train.json
  - test
    - test1.png
    - lksdjfl.png
    - ...
    - test.json

In [ ]:
#@title copy to local storage and extract
!cp -r '/content/drive/My Drive/dataset.7z' /content/dataset.7z
%cd /content/
!7z x /content/dataset.7z -oc:/content/

## Train config



[TannerGilbert/Tensorflow-Object-Detection-API-train-custom-Mask-R-CNN-model](https://github.com/TannerGilbert/Tensorflow-Object-Detection-API-train-custom-Mask-R-CNN-model) contains a small coco dataset, which is used as a test.

In [ ]:
#@title Downloading demo dataset
%cd /content/
!git clone https://github.com/TannerGilbert/Tensorflow-Object-Detection-API-train-custom-Mask-R-CNN-model object_detection

In [ ]:
#@title Register dataset
from detectron2.data.datasets import register_coco_instances
register_coco_instances("my_dataset_train2", {}, "/content/object_detection/images/train.json", "/content/object_detection/images/train/")
register_coco_instances("my_dataset_val2", {}, "/content/object_detection/images/test.json", "/content/object_detection/images/test/")

In [ ]:
#@title Activate tensorboard
# Look at training curves in tensorboard:
%load_ext tensorboard
%tensorboard --logdir "/content/drive/My Drive/detectron2"

# detectron2

In [ ]:
#@title (detectron2) mask_rcnn_X_101_32x8d_FPN_3x.yaml
import os
import numpy as np
import json
from detectron2.structures import BoxMode
from detectron2.engine import DefaultTrainer
from detectron2.config import get_cfg

cfg = get_cfg()

os.makedirs(cfg.OUTPUT_DIR, exist_ok=True)

cfg.merge_from_file("/content/detectron2/configs/COCO-InstanceSegmentation/mask_rcnn_X_101_32x8d_FPN_3x.yaml")

cfg.DATASETS.TRAIN = ("my_dataset_train2",)
cfg.DATASETS.TEST = ("my_dataset_val2",)
cfg.DATALOADER.NUM_WORKERS = 2

cfg.MODEL.WEIGHTS = ("/content/detectron2/configs/COCO-InstanceSegmentation/mask_rcnn_X_101_32x8d_FPN_3x.yaml")

cfg.INPUT.MAX_SIZE_TEST = 1700
cfg.INPUT.MAX_SIZE_TRAIN = 1700
cfg.SOLVER.IMS_PER_BATCH = 2   # increase this for more speed, but it will need more vram
cfg.SOLVER.BASE_LR = 0.00025  # pick a good LR
#cfg.SOLVER.BASE_LR = 0.000001  # pick a good LR /content/detectron2/configs/COCO-InstanceSegmentation/mask_rcnn_R_101_FPN_3x.yaml
cfg.SOLVER.MAX_ITER = 500000    # 300 iterations seems good enough for this toy dataset; you may need to train longer for a practical dataset
cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = 4096   # or 128 for example
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 2  # only has one class (ballon)
#cfg.SNAPSHOT_ITERS = 50 # when checkpoint gets created
cfg.SOLVER.CHECKPOINT_PERIOD = 5000

cfg.OUTPUT_DIR = "/content/drive/MyDrive/detectron2"
trainer = DefaultTrainer(cfg) 
#trainer.resume_or_load(resume=True)
trainer.train()

# detectron2-ResNeSt

In [ ]:
#@title (detectron2-ResNeSt) mask_cascade_rcnn_R_50_FPN_syncbn_1x.yaml

import os
import numpy as np
import json
from detectron2.structures import BoxMode
from detectron2.engine import DefaultTrainer
from detectron2.config import get_cfg

cfg = get_cfg()
cfg.OUTPUT_DIR = '/content/drive/MyDrive/detectron2'
os.makedirs(cfg.OUTPUT_DIR, exist_ok=True)

cfg.merge_from_file("/content/detectron2-ResNeSt/configs/COCO-InstanceSegmentation/mask_cascade_rcnn_R_50_FPN_syncbn_1x.yaml")

cfg.DATASETS.TRAIN = ("my_dataset_train2",)
cfg.DATASETS.TEST = ("my_dataset_val2",)
cfg.DATALOADER.NUM_WORKERS = 2

cfg.MODEL.WEIGHTS = ("/content/detectron2-ResNeSt/configs/COCO-InstanceSegmentation/mask_cascade_rcnn_R_50_FPN_syncbn_1x.yaml")

#1700x1700 ims_per_batch 2 max_mem: 12522M
#1800x1800 ims_per_batch 2 max_mem: 13303M
#3000x3000 ims_per_batch 1 max_mem: 7989M
cfg.INPUT.MAX_SIZE_TEST = 3000
cfg.INPUT.MAX_SIZE_TRAIN = 3000
cfg.SOLVER.IMS_PER_BATCH = 1   # increase this for more speed, but it will need more vram
cfg.SOLVER.BASE_LR = 0.00025  # pick a good LR

cfg.SOLVER.MAX_ITER = 500000    # 300 iterations seems good enough for this toy dataset; you may need to train longer for a practical dataset
cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = 4096   # or 128 for example
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 2  # only has one class (ballon)
#cfg.SNAPSHOT_ITERS = 50 # when checkpoint gets created
cfg.SOLVER.CHECKPOINT_PERIOD = 5000

trainer = DefaultTrainer(cfg) 
#trainer.resume_or_load(resume=True)
trainer.train()

In [ ]:
#@title (detectron2-ResNeSt) mask_cascade_rcnn_R_101_FPN_syncbn_1x.yaml

import os
import numpy as np
import json
from detectron2.structures import BoxMode
from detectron2.engine import DefaultTrainer
from detectron2.config import get_cfg

cfg = get_cfg()
cfg.OUTPUT_DIR = '/content/drive/MyDrive/detectron2'
os.makedirs(cfg.OUTPUT_DIR, exist_ok=True)

cfg.merge_from_file("/content/detectron2-ResNeSt/configs/COCO-InstanceSegmentation/mask_cascade_rcnn_R_101_FPN_syncbn_1x.yaml")

cfg.DATASETS.TRAIN = ("my_dataset_train2",)
cfg.DATASETS.TEST = ("my_dataset_val2",)
cfg.DATALOADER.NUM_WORKERS = 2

cfg.MODEL.WEIGHTS = ("/content/detectron2-ResNeSt/configs/COCO-InstanceSegmentation/mask_cascade_rcnn_R_101_FPN_syncbn_1x.yaml")

#1900x1900 ims_per_batch = 1 max_mem: 7539M
#2500x2500 ims_per_batch = 1 max_mem: 9193M
#3000x3000 ims_per_batch = 1 max_mem: 9236M
cfg.INPUT.MAX_SIZE_TEST = 3000
cfg.INPUT.MAX_SIZE_TRAIN = 3000
cfg.SOLVER.IMS_PER_BATCH = 1   # increase this for more speed, but it will need more vram
cfg.SOLVER.BASE_LR = 0.00025  # pick a good LR

cfg.SOLVER.MAX_ITER = 500000    # 300 iterations seems good enough for this toy dataset; you may need to train longer for a practical dataset
cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = 4096   # or 128 for example
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 2  # only has one class (ballon)
#cfg.SNAPSHOT_ITERS = 50 # when checkpoint gets created
cfg.SOLVER.CHECKPOINT_PERIOD = 5000

trainer = DefaultTrainer(cfg) 
#trainer.resume_or_load(resume=True)
trainer.train()

# centermask2

In [ ]:
#@title centermask2/train_net.py (registering dataset)
%%writefile /content/centermask2/train_net.py
# Copyright (c) Facebook, Inc. and its affiliates. All Rights Reserved
# Modified by Youngwan Lee (ETRI), 2020. All Rights Reserved.
from detectron2.data.datasets import register_coco_instances
register_coco_instances("my_dataset_train2", {}, "/content/object_detection/images/train.json", "/content/object_detection/images/train/")
register_coco_instances("my_dataset_val2", {}, "/content/object_detection/images/test.json", "/content/object_detection/images/test/")



import logging
import os
from collections import OrderedDict
import torch

import detectron2.utils.comm as comm
from detectron2.data import MetadataCatalog
from detectron2.engine import DefaultTrainer, default_argument_parser, default_setup, hooks, launch
from detectron2.evaluation import (
    # CityscapesInstanceEvaluator,
    # CityscapesSemSegEvaluator,
    # COCOEvaluator,
    COCOPanopticEvaluator,
    DatasetEvaluators,
    LVISEvaluator,
    PascalVOCDetectionEvaluator,
    SemSegEvaluator,
    verify_results,
)
from centermask.evaluation import (
    COCOEvaluator,
    CityscapesInstanceEvaluator,
    CityscapesSemSegEvaluator
)
from detectron2.modeling import GeneralizedRCNNWithTTA
from detectron2.checkpoint import DetectionCheckpointer
from centermask.config import get_cfg


class Trainer(DefaultTrainer):
    """
    This is the same Trainer except that we rewrite the
    `build_train_loader` method.
    """



    @classmethod
    def build_evaluator(cls, cfg, dataset_name, output_folder=None):
        """
        Create evaluator(s) for a given dataset.
        This uses the special metadata "evaluator_type" associated with each builtin dataset.
        For your own dataset, you can simply create an evaluator manually in your
        script and do not have to worry about the hacky if-else logic here.
        """
        if output_folder is None:
            output_folder = os.path.join(cfg.OUTPUT_DIR, "inference")
        evaluator_list = []
        evaluator_type = MetadataCatalog.get(dataset_name).evaluator_type
        if evaluator_type in ["sem_seg", "coco_panoptic_seg"]:
            evaluator_list.append(
                SemSegEvaluator(
                    dataset_name,
                    distributed=True,
                    output_dir=output_folder,
                )
            )
        if evaluator_type in ["coco", "coco_panoptic_seg"]:
            evaluator_list.append(COCOEvaluator(dataset_name, output_dir=output_folder))
        if evaluator_type == "coco_panoptic_seg":
            evaluator_list.append(COCOPanopticEvaluator(dataset_name, output_folder))
        if evaluator_type == "cityscapes_instance":
            assert (
                torch.cuda.device_count() >= comm.get_rank()
            ), "CityscapesEvaluator currently do not work with multiple machines."
            return CityscapesInstanceEvaluator(dataset_name)
        if evaluator_type == "cityscapes_sem_seg":
            assert (
                torch.cuda.device_count() >= comm.get_rank()
            ), "CityscapesEvaluator currently do not work with multiple machines."
            return CityscapesSemSegEvaluator(dataset_name)
        elif evaluator_type == "pascal_voc":
            return PascalVOCDetectionEvaluator(dataset_name)
        elif evaluator_type == "lvis":
            return LVISEvaluator(dataset_name, output_dir=output_folder)
        if len(evaluator_list) == 0:
            raise NotImplementedError(
                "no Evaluator for the dataset {} with the type {}".format(
                    dataset_name, evaluator_type
                )
            )
        elif len(evaluator_list) == 1:
            return evaluator_list[0]
        return DatasetEvaluators(evaluator_list)

    @classmethod
    def test_with_TTA(cls, cfg, model):
        logger = logging.getLogger("detectron2.trainer")
        # In the end of training, run an evaluation with TTA
        # Only support some R-CNN models.
        logger.info("Running inference with test-time augmentation ...")
        model = GeneralizedRCNNWithTTA(cfg, model)
        evaluators = [
            cls.build_evaluator(
                cfg, name, output_folder=os.path.join(cfg.OUTPUT_DIR, "inference_TTA")
            )
            for name in cfg.DATASETS.TEST
        ]
        res = cls.test(cfg, model, evaluators)
        res = OrderedDict({k + "_TTA": v for k, v in res.items()})
        return res



def setup(args):
    """
    Create configs and perform basic setups.
    """
    cfg = get_cfg()
    cfg.merge_from_file(args.config_file)
    cfg.merge_from_list(args.opts)
    cfg.freeze()
    default_setup(cfg, args)
    return cfg


def main(args):
    cfg = setup(args)

    if args.eval_only:
        model = Trainer.build_model(cfg)
        DetectionCheckpointer(model, save_dir=cfg.OUTPUT_DIR).resume_or_load(
            cfg.MODEL.WEIGHTS, resume=args.resume
        )
        res = Trainer.test(cfg, model)
        if cfg.TEST.AUG.ENABLED:
            res.update(Trainer.test_with_TTA(cfg, model))
        if comm.is_main_process():
            verify_results(cfg, res)
        return res

    """
    If you'd like to do anything fancier than the standard training logic,
    consider writing your own training loop or subclassing the trainer.
    """
    trainer = Trainer(cfg)
    trainer.resume_or_load(resume=args.resume)
    if cfg.TEST.AUG.ENABLED:
        trainer.register_hooks(
            [hooks.EvalHook(0, lambda: trainer.test_with_TTA(cfg, trainer.model))]
        )
    return trainer.train()


if __name__ == "__main__":
    args = default_argument_parser().parse_args()
    print("Command Line Args:", args)
    launch(
        main,
        args.num_gpus,
        num_machines=args.num_machines,
        machine_rank=args.machine_rank,
        dist_url=args.dist_url,
        args=(args,),
    )


In [ ]:
#@title (centermask2) Base-CenterMask-ResNet.yaml
%%writefile /content/centermask2/configs/centermask/Base-CenterMask-ResNet.yaml
MODEL:
  META_ARCHITECTURE: "GeneralizedRCNN"
  BACKBONE:
    NAME: "build_fcos_resnet_fpn_backbone"
  RESNETS:
    OUT_FEATURES: ["res3", "res4", "res5"]
  FPN:
    IN_FEATURES: ["res3", "res4", "res5"]
  PROPOSAL_GENERATOR:
    NAME: "FCOS"
  FCOS:
    POST_NMS_TOPK_TEST: 50
  # PIXEL_MEAN: [102.9801, 115.9465, 122.7717]
  MASK_ON: True
  MASKIOU_ON: True
  ROI_HEADS:
    NAME: "CenterROIHeads"
    IN_FEATURES: ["p3", "p4", "p5"]
  ROI_MASK_HEAD:
    NAME: "SpatialAttentionMaskHead"
    ASSIGN_CRITERION: "ratio"
    NUM_CONV: 4
    POOLER_RESOLUTION: 14
DATASETS:
  TRAIN: ("my_dataset_train2",)
  TEST: ("my_dataset_val2",)
SOLVER:
  IMS_PER_BATCH: 8
  BASE_LR: 0.01  # Note that RetinaNet uses a different default learning rate
  STEPS: (60000, 80000)
  MAX_ITER: 90000
INPUT:
  MIN_SIZE_TRAIN: (640, 672, 704, 736, 768, 800)

In [ ]:
#@title (centermask2) Base-CenterMask-ResNet.yaml
%cd /content/centermask2/
!python train_net.py --num-gpus 1 --config /content/centermask2/configs/centermask/centermask_R_50_FPN_ms_3x.yaml

In [ ]:
#@title restart kernel if something goes wrong
import os
os._exit(00)

## Run tests with model



In [ ]:
#@title load model
cfg.MODEL.WEIGHTS = '/content/drive/My Drive/model_0139999.pth'
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.30   # set the testing threshold for this model
cfg.DATASETS.TEST = ("my_dataset_val2", )
predictor = DefaultPredictor(cfg)

In [ ]:
#@title do detection and display result
from detectron2.utils.visualizer import ColorMode
balloon_metadata = MetadataCatalog.get("my_dataset_train2")

im = cv2.imread("/content/picture.jpg")
outputs = predictor(im)
v = Visualizer(im[:, :, ::-1],
                metadata=balloon_metadata, 
                scale=0.5, 
                instance_mode=ColorMode.IMAGE_BW   # remove the colors of unsegmented pixels. This option is only available for segmentation models
)
v = v.draw_instance_predictions(outputs["instances"].to("cpu"))
cv2_imshow(v.get_image()[:, :, ::-1])

In [ ]:
#@title AP metric
from detectron2.evaluation import COCOEvaluator, inference_on_dataset
from detectron2.data import build_detection_test_loader
evaluator = COCOEvaluator("my_dataset_val2", cfg, False, output_dir="./output/")
val_loader = build_detection_test_loader(cfg, "my_dataset_val2")
inference_on_dataset(trainer.model, val_loader, evaluator)
# another equivalent way is to use trainer.test